In [1]:
import sys
sys.path.append('../')

import numpy as np
import pandas as pd
import tempfile
import json
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

import logging
logging.getLogger().setLevel(logging.INFO)

from banditpylib import trials_to_dataframe
from banditpylib.arms import GaussianArm
from banditpylib.bandits import MultiArmedBandit
from banditpylib.protocols import SinglePlayerProtocol
from banditpylib.learners.mab_fcbai_learner import ExpGap, LilUCBHeuristic, TrackAndStop
from banditpylib.utils import argmax_or_min_tuple, argmax_or_min, argmax_or_min_tuple_second

In [2]:
confidence = 0.95
means =  [0.1, 0.4, 0.7]
#means = [0.5, 0.45, 0.43, 0.4] #w∗(µ1) = [0.417 0.390 0.136 0.057]
#means =  [0.3, 0.21, 0.2, 0.19, 0.18] #w∗(µ2) = [0.336, 0.251, 0.177, 0.132, 0.104]
max_pulls=3000
forced_exp_pulls=20
std=1

arms = [GaussianArm(mu=mean, std=std) for mean in means]
bandit = MultiArmedBandit(arms=arms)
learners = [
            #ExpGap(arm_num=len(arms), confidence=confidence, threshold=3, name='Exponential-Gap Elimination'),
            LilUCBHeuristic(arm_num=len(arms), confidence=confidence, name='Heuristic lilUCB'),
            TrackAndStop(arm_num=len(arms), confidence=confidence, tracking_rule="D", 
                        max_pulls=max_pulls, forced_exp_pulls=forced_exp_pulls, std=std, name='Track and stop')
           ]

# For each setup, we run 20 trials
trials = 20
temp_file = tempfile.NamedTemporaryFile()

In [ ]:
game = SinglePlayerProtocol(bandit=bandit, learners=learners)
# Start playing the game
# Add `debug=True` for debugging purpose
game.play(trials=trials, output_filename=temp_file.name)

INFO:absl:start Heuristic lilUCB's play with multi_armed_bandit
INFO:absl:Heuristic lilUCB's play with multi_armed_bandit runs 7.26 seconds.
INFO:absl:start Track and stop's play with multi_armed_bandit


In [ ]:
trials_df = trials_to_dataframe(temp_file.name)

In [ ]:
trials_df.head()

In [ ]:
trials_df['confidence'] = confidence

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
sns.barplot(x='confidence', y='total_actions', hue='learner', data=trials_df)
plt.ylabel('pulls')
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))